1. Viết các hàm đọc và lấy thông tin từ tất cả các file xml hoặc json trong folder theo
yêu cầu sau: (một số configuration và data đã được định nghĩa trước và đính kèm vào
link file notebook bên dưới)

(a) Viết hàm readxml(path) đọc toàn bộ file xml và lấy các thông tin sau "filename", và thông
tin của các "object" bao gồm "name", "xmin", "ymin", "xmax", "ymax" và trả về một
dictionary với mỗi item gồm key là tên đường dẫn ảnh, value là một list chứa thông tin của
các "object"
• Input: path đường dẫn file chứa ảnh (.jpg) và file xml (.xml).
• Output: một dictionary
– key: đường dẫn đến file ảnh
– value: list với mỗi element là ’name’ và tuple(xmin, ymin, xmax, ymax)

(b) Viết hàm readjon(path) đọc toàn bộ file json và lấy các thông tin sau "imagePath", và thông
tin của "shapes" bao gồm "label", và "points". Sau đó trả về một dictionary với mỗi item
gồm key là tên đường dẫn ảnh, value là một list chứa thông tin của các "shapes"
• Input: Đường dẫn file chứa ảnh (.jpg) và file json (.json).
• Output: một dictionary
– key: path đường dẫn đến file ảnh
– value: list với mỗi element là ’label’ và list points

Hướng dẫn cách làm bài:
• Toàn bộ bài tập sẽ thực hiện theo template notebook này (link)
Hình 3: Cell làm bài tập 1a và 1b
• Các bạn vào teamplate notebook ở trên, chạy hết các cell ở phần có tên là Exercise1 để
download các file, data và giải nén data, cũng như import các thư viện cần thiết.
• Tiếp theo các bạn tìm đến các cell như hình 3 ở trên để hoàn thành bài tập
• Các bạn phải đọc nội dung các file trong path1a, và path1b để biết folder chứa cả
ảnh và file xml hoặc json. Đồng thời đọc nội dung các file để biết cấu trúc và vị trí các thông
tin mà mình cần lấy theo yêu cầu của đề .
• Sau đó viết nội dung cho hàm readxml (nhận input path1a) và readjson (nhận input path1b)
và return về dictionary kết quả
Đối với key của dictionary là đường dẫn đến file ảnh (file ảnh và file xml hoặc json sẽ cùng
tên nhưng khác đuôi mở rộng .jpg, .xml hoặc .json) các bạn sẽ phải gắn đường dẫn đến folder
chứa ảnh và tên file ảnh. Tên file ảnh chính là "filename" (1a) và "imagePath" (1b). Để thu
được đường dẫn đến ảnh cũng chính là key
• lưu ý 1a: xmin, ymin, xmax, ymax khi lấy ra sẽ có dạng string các bạn nhớ ép kiểu về dạng
int
• lưu ý 2a: các bạn lấy trực tiếp "points" là list và không cần xử lý gì thêm.
• Sau khi có được dictionary, các bạn convert sang iterator và dùng hàm có sẵn (draw_one_img
với is_from_xml=True (1a) và is_from_xml=False (1b)) để vẽ các kết quả lên ảnh như ví
dụ bên dưới



In [2]:
%pip install gdown
import gdown
!gdown --id 1B6YL5voMkc9YqPp9wKx06KOTuGRiuXQM

In [ ]:
##
# EXERCISE 1:
#
import os
import json
import xml.etree.ElementTree as ET
from P1 import draw_one_img

def readxml(path):
    results = {}

    for fname in 